In [3]:
import os
from openslide import OpenSlide, open_slide, deepzoom
import numpy as np
from scipy import linalg
import napari
from skimage.io import imread
from dask import delayed
import dask.array as da
#os.chdir("../..")


In [2]:
sm_path = "D:/repos/swe22_final/sm.svs"
#slide = OpenSlide(sm_path)

In [4]:
def svs2dask_array_pyramid(img, gen, level, tile_size=1000, overlap=0, remove_last=True, allow_unknown_chunksizes=False):	
	max_level = gen.level_count - 1 - 2 * level
	n_tiles_x, n_tiles_y = gen.level_tiles[max_level]
	get_tile = lambda level,i,j: np.array(gen.get_tile(level,(i,j))).transpose((1,0,2))
	sample_tile = get_tile(max_level,0,0)
	sample_tile_shape = sample_tile.shape
	dask_get_tile = delayed(get_tile, pure=True)
	arr = (da.concatenate([da.concatenate([da.from_delayed(dask_get_tile(max_level,i,j),sample_tile_shape,np.uint8) for j in range(n_tiles_y)],allow_unknown_chunksizes=allow_unknown_chunksizes,axis=1) for i in range(n_tiles_x )],allow_unknown_chunksizes=allow_unknown_chunksizes))#.transpose([1,0,2]))

	return arr

sm_path = "D:/repos/swe22_final/sm.svs"
img=open_slide(sm_path)
gen=deepzoom.DeepZoomGenerator(img)
lvls = len(open_slide(sm_path).level_dimensions)
viewer = napari.Viewer() 
viewer.add_image( [svs2dask_array_pyramid(img, gen, level) for level in range(lvls)], contrast_limits=[0,255])


<Image layer 'Image' at 0x1f8d2c632b0>

In [ ]:
img=open_slide(sm_path)
gen=deepzoom.DeepZoomGenerator(img, tile_size=1000, overlap=0, limit_bounds=True)
gen.level_dimensions

In [ ]:
def getHEDfromRGB():
    
    # Conversion matrix for RGB-HED
    
    # Haematoxylin-Eosin-DAB colorspace
    # From original Ruifrok's paper: A. C. Ruifrok and D. A. Johnston,
    # "Quantification of histochemical staining by color deconvolution,"
    # Analytical and quantitative cytology and histology / the International
    # Academy of Cytology [and] American Society of Cytology, vol. 23, no. 4,
    # pp. 291-9, Aug. 2001.
    rgb_from_hed = np.array([[0.65, 0.70, 0.29],
                            [0.07, 0.99, 0.11],
                            [0.27, 0.57, 0.78]])
    hed_from_rgb = linalg.inv(rgb_from_hed)

    return


In [ ]:
viewer = napari.Viewer()
#t, c, z, y_slice, x_slice